In [1]:
print('Loading Library')

Loading Library


In [ ]:
import boto3
from pydub import AudioSegment
import textwrap

# Define file paths
input_text_file = "C:/Users/bciez/Documents/Basilio/youtube/Historias/LaLloronaDiezHistoria.txt"  # Text file containing the story
output_audio_path = "C:/Users/bciez/Documents/Basilio/youtube/Historias/LaLloronaDiezHistoria.mp3"

# Load story text from the file
with open(input_text_file, "r", encoding="utf-8") as file:
    story_text = file.read()

# Manually define AWS credentials
aws_access_key_id = "access_key"
aws_secret_access_key = "secret_key"
region_name = "us-east-1"  # Change if needed

# Initialize AWS Polly client
polly_client = boto3.client(
    "polly",
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=region_name
)

# ✅ Split text into smaller chunks (~2,900 characters each)
text_chunks = textwrap.wrap(story_text, width=2900, break_long_words=False, replace_whitespace=False)

# ✅ Generate audio for each chunk
audio_files = []
for idx, chunk in enumerate(text_chunks):
    ssml_text = f"""
    <speak>
        <prosody rate="slow">
            {chunk}
        </prosody>
    </speak>
    """

    response = polly_client.synthesize_speech(
        Text=ssml_text,
        OutputFormat="mp3",
        VoiceId="Enrique",  
        TextType="ssml"
    )

    # Save each chunk as a separate MP3 file
    file_path = f"C:/Users/bciez/Documents/Basilio/youtube/llorona_part{idx}.mp3"
    with open(file_path, "wb") as file:
        file.write(response["AudioStream"].read())
    
    audio_files.append(file_path)

# ✅ Merge all audio parts with consistent encoding
final_audio = None
for file in audio_files:
    segment = AudioSegment.from_mp3(file).set_frame_rate(44100).set_channels(2)  # Ensure standard 44.1kHz stereo
    if final_audio is None:
        final_audio = segment
    else:
        final_audio += segment

# ✅ Export final audio file in MP3 format
if final_audio:
    final_audio.export(output_audio_path, format="mp3", bitrate="192k")

print(f"✅ Final audio saved as '{output_audio_path}'")


✅ Final audio saved as 'C:/Users/bciez/Documents/Basilio/youtube/Historias/LaLloronaDiezHistoria.mp3'
